In [2]:
import numpy as np
import pandas as pd
import os
import datetime
import sys
import re
# import json
import time
# import zipfile
# import cpca  # needs manually installing
from sklearn import preprocessing
import pymysql
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pylab as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import os
import datetime
import sys
import re
import json
import time
import zipfile
# import cpca  # needs manually installing
from sklearn import preprocessing


unique = lambda x:len(set(x))  # 唯一值的个数

def read_file(f,**kw):
    '''读文件'''
    try:
        tmp = pd.read_csv(f,**kw)
    except:
        try:
            tmp = pd.read_excel(f,**kw)
        except:
            try:
                tmp = pd.read_table(f,**kw)
            except:
                print('Wrong file: ',f)
                return None
    return tmp

def walk_zip_files(path,pattern_zip='.*.zip',pattern_file='.*trx.*.csv',colnum=False,
                   pattern_date='201[89].\d{2}.\d{2}',**kw):
    '''
    解析文件夹下所有zip里的符合pattern的文件
    参数：
    -- path：文件路径
    -- pattern_zip：zip文件正则表达式
    -- pattern_file：文本文件正则表达式
    -- colnum：文件列数；int；默认False代表不用指定列数来过滤文件
    -- pattern_date：文件日期正则表达式，False表示不取日期列
    -- **kw：文件读取参数
    返回：
    -- 符合条件的所有文件合并后的dataframe
    '''
    import zipfile
    import re
    trx = None

    for dirpath,dirnames,filenames in os.walk(path):
        for file in filenames:
            if re.search(pattern_zip,file):
                fullpath=os.path.join(dirpath,file)
                with zipfile.ZipFile(fullpath, "r") as z:
                    for i in z.namelist():
                        if re.search(pattern_file,i):
                            f = z.open(i)
#                             print('reading: ',fullpath)
                            tmp = read_file(f,**kw)
                            try:
                                print(fullpath,i,'行列数：',tmp.shape)
                                if pattern_date:
                                    try:
                                        tmp['file_date'] = re.findall(string=fullpath,pattern=pattern_date)[0]
                                    except: pass
                                if colnum:
                                    if tmp.shape[1] == colnum:
                                        trx = pd.concat([trx,tmp],axis=0,ignore_index=True)
                                else:
                                    trx = pd.concat([trx,tmp],axis=0,ignore_index=True)
                            except: pass           
    return trx

def walk_files(path,pattern_file='.*loan.*csv',colnum=False,pattern_date='201[89].\d{2}.\d{2}',**kw):
    '''
    提取指定path目录下所有符合pattern正则式文件的内容
    参数：
    -- path：文件路径
    -- pattern_file：文件名正则表达式
    -- colnum：列数;int；默认False代表不用指定列数来过滤文件
    -- pattern_date：文件日期正则表达式，从文本文件路径中得到
    -- **kw：文件读取参数
    返回：
    -- 符合条件的所有文件合并后的dataframe
    '''
    import re
    data = None
    for dirpath,dirnames,filenames in os.walk(path):
        for file in filenames:
            if re.search(pattern_file,file):
                fullpath=os.path.join(dirpath,file)
#                 print('reading: ',fullpath)
                tmp = read_file(fullpath,**kw)
                try:
                    print(fullpath,'行列数：',tmp.shape)
                    if pattern_date:
                        ## 一般路径或文件名会含有文件上传时间，匹配pattern_date正则来解析文件日期
                        try:
                            tmp['file_date'] = re.findall(string=fullpath,pattern=pattern_date)[0]
                        except:pass
                    if colnum:
                        if tmp.shape[1] == colnum:
                            data = pd.concat([data,tmp],axis=0,ignore_index=True)
                    else:
                        data = pd.concat([data,tmp],axis=0,ignore_index=True)
                except: pass
    return data



def convert_time(time):
    '''字符转换为时间'''
    if not time:
        return
    try:
        return datetime.datetime.strptime(str(time), "%Y-%m-%d")
    except:
        try:
            return datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S")
        except:
            try:
                return datetime.datetime.strptime(str(time), "%Y%m%d%H%M%S")
            except:
                try:
                    return datetime.datetime.strptime(str(time), "%Y%m%d")
                except:
                    try:
                         return datetime.datetime.strptime(str(time), "%Y/%m/%d")
                    except:
                        try:
                            return datetime.datetime.strptime(str(time), "%Y/%m/%d %H:%M")
                        except:
                            try:
                                return datetime.datetime.strptime(str(time), "%Y/%m/%d %H:%M:%S")
                            except:
                                try:
                                    return datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M")
                                except:
                                    try:
                                        return datetime.datetime.strptime(str(time), "%Y-%m-%dT%H:%M:%S.000+08:00")
                                    except:
                                        print("Wrong date format : %s " % time)
                                        return None

def convert_time_col(df,pattern_date='(time|date|ddl)',suffix=''):
    '''
    对df含有时间类型的列，其字符串转换成标准时间
    参数：
    -- df:dataframe
    -- pattern_date:时间列表达式
    -- suffix：时间列添加后缀
    '''
    columns = df.columns
    # 时间类型
    time_cols = columns[columns.str.lower().str.contains(pat=pattern_date)]
    for time_col in time_cols:
        df[str(time_col)+suffix] = df[time_col].apply(convert_time)   
    return df

# 标准分量化

In [40]:
supp_rank.columns,supp_rank.columns[0]

(Index(['下单总次数', '订单完成总次数', '订单完成率', '完成交易总金额', '优惠次数', '优惠总金额', '品类个数',
        '周交易密度', '周交易金额', '月交易密度', '月交易金额', '过去30天交易次数', '过去30天交易金额',
        '过去60天交易次数', '过去60天交易金额', '过去90天交易次数', '过去90天交易金额', '过去180天交易次数',
        '过去180天交易金额', '订单取消总次数', '平均下单周期（天）', '发货缺口次数', '发货缺口金额', '收货缺口次数',
        '收货缺口金额', '交易间隔时间的均值（天）', '交易间隔时间的标准差（天）', '最大交易间隔时间（天）'],
       dtype='object'), '下单总次数')

In [43]:
weight_value=[]
for i in supp_rank.columns:
    weight_value.append(supp_weight[i])

In [44]:
weight_value

[2,
 5,
 5,
 8,
 2,
 2,
 5,
 3,
 3,
 3,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2]

In [52]:
def get_weights(weight_file,**kw):
    df = pd.read_excel(weight_file,**kw)
    weights = dict(zip(df['指标'],df['权重']))
    return weights

def get_score(rank,dim_name='机构',weights=[],level_num=4): 
    rank['总分'] = round((rank*weights).sum(axis=1)/np.sum(weights,axis=0),4)
    result = rank.sort_values([u"总分"], ascending = False)#.reset_index()
    
    baselines = []
    levels_prop = np.linspace(0,1,level_num+1)
    for prop in levels_prop:
        baseline = np.percentile(result[u"总分"].values, prop)
        baselines.append(baseline)
        print("current threshold is %s%%, score is %s" % (prop, baseline))
    baselines[0] = baselines[0] - 0.000001
    print('baselines:',baselines)
    rank['排名'] = rank['总分'].rank(ascending=False,method='dense').apply(int)
    rank = rank.sort_values('排名')
    rank['{}评级'.format(dim_name)] = pd.cut(rank['总分'],bins=baselines,
                               right=True,labels=['D','C','B','A'])
    return rank


def suppvalue2rank(df,groupby_col=['company_ID_num'],weights={}):       
    high_is_better = [
                    '下单总次数',
                    '订单完成总次数',
                    '订单完成率',
                    '完成交易总金额',
                    '优惠次数',
                    '优惠总金额',
                    '品类个数',
                    '周交易密度',
                    '周交易金额',
                    '月交易密度',
                    '月交易金额',
                    '过去30天交易次数',
                    '过去30天交易金额',
                    '过去60天交易次数',
                    '过去60天交易金额',
                    '过去90天交易次数',
                    '过去90天交易金额',
                    '过去180天交易次数',
                    '过去180天交易金额',
                     ]

    low_is_better = [
                    '订单取消总次数',
                    '平均下单周期（天）',
                    '发货缺口次数',
                    '发货缺口金额',
                    '收货缺口次数',
                    '收货缺口金额',
                    '交易间隔时间的均值（天）',
                    '交易间隔时间的标准差（天）',
                    '最大交易间隔时间（天）',
                    ]
    
    total_num = float(df.shape[0])
    min_max_scaler = preprocessing.MinMaxScaler()
    
    result = None
    for col in high_is_better:
        tmp = df[[col]].fillna(float('-inf')).sort_values(by=[col], ascending=True).rank(ascending=True,method="dense")
        tmp[col] = min_max_scaler.fit_transform(tmp)
        tmp[col] = round(tmp[col], 4)
        result = pd.concat([result, tmp],axis=1,)
        

    for col in low_is_better:
        tmp = df[[col]].fillna(float('inf')).sort_values(by=[col], ascending=True).rank(ascending=False,method="dense")
        tmp[col] = min_max_scaler.fit_transform(tmp)
        tmp[col] = round(tmp[col], 4)
        result = pd.merge(result, tmp, left_index=True, right_index=True, how="left")
        
    return result

In [7]:
supp_weight = get_weights(r"E:/360MoveData/Users/chenyao/Desktop/中驰车福量子科技评分权重表_订单交易版.xlsx")

In [8]:
supp_weight

{'下单总次数': 2,
 '订单取消总次数': 2,
 '订单完成总次数': 5,
 '订单完成率': 5,
 '平均下单周期（天）': 2,
 '完成交易总金额': 8,
 '优惠次数': 2,
 '优惠总金额': 2,
 '发货缺口次数': 2,
 '发货缺口金额': 2,
 '收货缺口次数': 2,
 '收货缺口金额': 2,
 '品类个数': 5,
 '周交易密度': 3,
 '周交易金额': 3,
 '月交易密度': 3,
 '月交易金额': 3,
 '过去30天交易次数': 5,
 '过去30天交易金额': 5,
 '过去60天交易次数': 5,
 '过去60天交易金额': 5,
 '过去90天交易次数': 5,
 '过去90天交易金额': 5,
 '过去180天交易次数': 5,
 '过去180天交易金额': 5,
 '交易间隔时间的均值（天）': 2,
 '交易间隔时间的标准差（天）': 3,
 '最大交易间隔时间（天）': 2}

In [9]:
company= pd.read_excel('E:/360MoveData/Users/chenyao/Desktop/中驰车福订单分析表_new.xlsx')

In [10]:
company.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4936 entries, 0 to Row_sum
Data columns (total 31 columns):
企业名称             4936 non-null object
最早交易时间           4936 non-null datetime64[ns]
最近交易时间           4936 non-null datetime64[ns]
下单总次数            4936 non-null float64
订单取消总次数          4936 non-null int64
订单完成总次数          4936 non-null float64
订单完成率            4936 non-null float64
平均下单周期（天）        4936 non-null float64
完成交易总金额          4936 non-null float64
优惠次数             4936 non-null int64
优惠总金额            4936 non-null int64
发货缺口次数           4936 non-null float64
收货缺口次数           4936 non-null float64
发货缺口金额           4936 non-null float64
收货缺口金额           4936 non-null float64
品类个数             4936 non-null float64
周交易密度            4936 non-null float64
周交易金额            4936 non-null float64
月交易密度            4936 non-null float64
月交易金额            4936 non-null float64
过去30天交易次数        4936 non-null float64
过去30天交易金额        4936 non-null float64
过去60天交易次数        4936 non-null

In [22]:
loan=pd.read_csv('E:/360MoveData/Users/chenyao/Desktop/loan_info.csv')

In [53]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345 entries, 0 to 1344
Data columns (total 22 columns):
企业名称              1345 non-null object
最早贷款时间            1345 non-null object
最近贷款时间            1345 non-null object
贷款申请总次数           1345 non-null int64
贷款取消总次数           1345 non-null float64
贷款终止总次数           1345 non-null float64
贷款完成总次数           1345 non-null float64
贷款逾期总次数           1345 non-null float64
贷款仍活跃总次数          1345 non-null float64
申请贷款完成率           1345 non-null float64
贷款总金额             1345 non-null float64
逾期率               1345 non-null float64
周贷款次数             1345 non-null float64
周贷款金额             1345 non-null float64
月贷款次数             1345 non-null float64
月贷款金额             1345 non-null float64
贷款账期的均值（天）        1234 non-null float64
贷款账期的标准差（天）       1092 non-null float64
贷款回款期的均值（天）       1234 non-null float64
贷款回款期的标准差（天）      1092 non-null float64
company_name      1345 non-null object
company_ID_num    1326 non-null object
dtypes: float64(16),

In [58]:
company_loan=company.merge(loan,on='company_name',how='inner')

In [66]:
company_loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1329 entries, 0 to 1328
Data columns (total 52 columns):
企业名称_x              1329 non-null object
最早交易时间              1329 non-null object
最近交易时间              1329 non-null object
下单总次数               1329 non-null int64
订单取消总次数             1329 non-null float64
订单完成总次数             1329 non-null float64
订单完成率               1329 non-null float64
平均下单周期（天）           1329 non-null float64
完成交易总金额             1329 non-null float64
优惠次数                1329 non-null float64
优惠总金额               1329 non-null float64
发货缺口次数              1329 non-null float64
发货缺口金额              1329 non-null float64
收货缺口次数              1329 non-null float64
收货缺口金额              1329 non-null float64
品类个数                1329 non-null float64
周交易密度               1329 non-null float64
周交易金额               1329 non-null float64
月交易密度               1329 non-null float64
月交易金额               1329 non-null float64
过去30天交易次数           1329 non-null float64
过去30天交易金额       

In [12]:
del company_loan['企业名称_y']

NameError: name 'company_loan' is not defined

In [67]:
company_loan.rename(columns = {"企业名称_x": "企业名称", "company_ID_num_x":"company_ID_num"},  inplace=True)

In [68]:
company_loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1329 entries, 0 to 1328
Data columns (total 52 columns):
企业名称              1329 non-null object
最早交易时间            1329 non-null object
最近交易时间            1329 non-null object
下单总次数             1329 non-null int64
订单取消总次数           1329 non-null float64
订单完成总次数           1329 non-null float64
订单完成率             1329 non-null float64
平均下单周期（天）         1329 non-null float64
完成交易总金额           1329 non-null float64
优惠次数              1329 non-null float64
优惠总金额             1329 non-null float64
发货缺口次数            1329 non-null float64
发货缺口金额            1329 non-null float64
收货缺口次数            1329 non-null float64
收货缺口金额            1329 non-null float64
品类个数              1329 non-null float64
周交易密度             1329 non-null float64
周交易金额             1329 non-null float64
月交易密度             1329 non-null float64
月交易金额             1329 non-null float64
过去30天交易次数         1329 non-null float64
过去30天交易金额         1329 non-null float64
过去60天交易次数         

In [53]:
supp_rank = suppvalue2rank(company.set_index(['企业名称']),weights=supp_weight)

In [54]:
supp_rank

,下单总次数,订单完成总次数,订单完成率,完成交易总金额,优惠次数,优惠总金额,品类个数,周交易密度,周交易金额,月交易密度,...,过去180天交易金额,订单取消总次数,平均下单周期（天）,发货缺口次数,发货缺口金额,收货缺口次数,收货缺口金额,交易间隔时间的均值（天）,交易间隔时间的标准差（天）,最大交易间隔时间（天）
3003294,0.0057,0.0073,1.0000,0.1905,0.0,0.0,0.0229,0.0134,0.2545,0.0099,...,0.0051,0.0,0.8234,0.9861,0.2918,1.000,0.8529,0.4755,0.4216,0.9261
3003295,0.0014,0.0029,1.0000,0.0989,0.0,0.0,0.0153,0.0067,0.1284,0.0049,...,0.0507,0.0,0.6228,1.0000,0.1882,1.000,0.8529,0.2746,0.0000,0.9340
3005833,0.0042,0.0058,1.0000,0.0056,0.0,0.0,0.0229,0.0268,0.0052,0.0197,...,0.0051,0.0,1.0000,0.9931,0.3636,1.000,0.8529,1.0000,0.0000,1.0000
3005889,0.0000,0.0015,1.0000,0.4613,0.0,0.0,0.0076,0.0067,0.5269,0.0049,...,0.0051,0.0,1.0000,1.0000,0.1882,1.000,0.8529,0.0000,0.0000,0.0000
3006282,0.0184,0.0203,1.0000,0.3218,0.0,0.0,0.0153,0.0134,0.2527,0.0099,...,0.0051,0.0,0.4760,1.0000,0.1882,1.000,0.8529,0.2983,0.4104,0.8417
3006563,0.6167,0.6125,0.8543,0.2612,0.0,0.0,0.0305,0.0604,0.0650,0.1379,...,0.0051,0.0,0.9701,0.9931,0.1845,1.000,0.8529,0.7270,0.8696,0.9314
3006702,0.0311,0.0334,1.0000,0.2410,0.0,0.0,0.0382,0.0134,0.1647,0.0148,...,0.0051,0.0,0.5898,1.0000,0.1882,1.000,0.8529,0.2089,0.2069,0.6755
3006725,0.0071,0.0073,0.3028,0.1193,0.0,0.0,0.0229,0.0067,0.1089,0.0049,...,0.0051,0.0,0.1467,1.0000,0.1882,1.000,0.8529,0.0781,0.1308,0.6623
3006752,0.0198,0.0218,1.0000,0.2113,0.0,0.0,0.0305,0.0134,0.1817,0.0148,...,0.0051,0.0,0.7275,1.0000,0.1882,1.000,0.8529,0.5331,0.4237,0.8549
3006820,0.0877,0.0668,0.2048,0.2332,0.0,0.0,0.0458,0.0336,0.1568,0.0394,...,0.0051,0.0,0.8772,1.0000,0.1882,1.000,0.8529,0.3619,0.2503,0.6834


In [55]:
supp_score = get_score(supp_rank,dim_name='企业名称',weights=weight_value)

current threshold is 0.0%, score is 0.0673
current threshold is 0.25%, score is 0.07630125
current threshold is 0.5%, score is 0.07947000000000001
current threshold is 0.75%, score is 0.0824025
current threshold is 1.0%, score is 0.0831
baselines: [0.067299, 0.07630125, 0.07947000000000001, 0.0824025, 0.0831]


In [51]:
supp_score

,下单总次数,订单完成总次数,订单完成率,完成交易总金额,优惠次数,优惠总金额,品类个数,周交易密度,周交易金额,月交易密度,...,发货缺口次数,发货缺口金额,收货缺口次数,收货缺口金额,交易间隔时间的均值（天）,交易间隔时间的标准差（天）,最大交易间隔时间（天）,总分,排名,企业名称评级
QYYG00047047962,0.9929,0.9927,0.9987,0.9993,0.0,0.0,0.3893,0.7852,0.9935,0.9360,...,0.0069,0.8991,1.0,0.8529,0.9790,0.9682,0.9129,0.8483,1,NaN
QYYG00154154408,0.9576,0.9594,1.0000,0.7780,0.0,0.0,0.9466,0.6242,0.5323,0.8522,...,0.1667,0.8355,1.0,0.8529,0.9618,0.8959,0.8259,0.8165,2,NaN
QYYG00177177712,0.3479,0.3556,1.0000,0.6703,0.0,0.0,0.0076,0.5369,0.6575,0.8227,...,0.9861,0.5082,1.0,0.8529,0.9505,0.9720,0.9868,0.7184,3,NaN
JXS570571126,0.8614,0.8665,0.9950,0.8932,0.0,0.0,0.0687,0.3289,0.6259,0.4384,...,1.0000,0.1882,1.0,0.8529,0.6295,0.6364,0.8285,0.6795,4,NaN
JXS132142318,0.7129,0.7170,0.9899,0.9163,0.0,0.0,0.0763,0.1342,0.5997,0.1773,...,0.9861,0.4609,1.0,0.8529,0.5794,0.5357,0.7520,0.6780,5,NaN
JXS580608972,0.8755,0.8810,1.0000,0.9103,0.0,0.0,0.0534,0.3893,0.6689,0.5074,...,1.0000,0.1882,1.0,0.8529,0.6796,0.5855,0.7573,0.6775,6,NaN
QXC5013656689,0.9972,0.9971,0.8819,0.6943,0.0,0.0,0.9924,0.5638,0.3360,0.8916,...,0.0208,0.6809,1.0,0.8529,0.9925,0.9824,0.8285,0.6749,7,NaN
QXC6346974943,0.9788,0.9797,0.8153,0.6871,0.0,0.0,0.7405,0.5570,0.3786,0.8867,...,0.2431,0.6464,1.0,0.8529,0.9892,0.9678,0.8364,0.6734,8,NaN
JXS1219220811,0.2136,0.1567,0.1922,0.8751,0.0,0.0,0.0305,0.3691,0.9937,0.5074,...,1.0000,0.1882,1.0,0.8529,0.9882,0.9900,0.9894,0.6687,9,NaN
QYYG00162162598,0.9420,0.9042,0.1847,0.6497,0.0,0.0,0.2519,0.5772,0.4505,0.6749,...,0.3750,0.6455,1.0,0.8529,0.9478,0.8851,0.8496,0.6665,10,NaN


In [23]:
supp_score.to_excel('E:/360MoveData/Users/chenyao/Desktop/中驰车福量子科技企业评分结果.xlsx')